In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib as plt
import itertools as it
from analysis_utils import *

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()
%load_ext rpy2.ipython

# Setup

In [ ]:
EXPERIMENT = 4

os.makedirs(f'stats/{EXPERIMENT}', exist_ok=True)
def write_tex(name, tex):
    file = f"stats/{EXPERIMENT}/{name}.tex"
    with open(file, "w+") as f:
        f.write(str(tex) + r"\unskip")
    print(f'wrote "{tex}" to "{file}"')

data = get_data(EXPERIMENT)
pdf = data['participants'].set_index('pid').copy()
full_pdf = pdf.copy()
mdf = data['trials'].set_index('pid').copy()
mdf.trial_time /= 1000
mdf['clicked'] = mdf.n_clicks > 0

if EXPERIMENT == 3:
    pdf.query('completed_1', inplace=True)
else:
    pdf.query('completed', inplace=True)
    

if EXPERIMENT == 4:
    fb_namer = {
        (0, 0): 'none',
        (1, 0): 'info_only',
        (0, 1): 'reward_only',
        (1, 1): 'both'
    }
    pdf['feedback'] = pdf.apply(lambda row: fb_namer[row.information, row.reward], axis=1)
    mdf['information'] = pdf.information
    mdf['reward'] = pdf.reward
    fb_order = list(fb_namer.values())
else:
    fb_order = [fb for fb in ['none', 'action', 'meta'] if fb in set(pdf.feedback)]

## Demographics

In [ ]:
pids = list(pdf.index.unique())
sdf = data['survey'].query('pid == @pids').copy()
if not isinstance(sdf.responses.iloc[0], dict):
    sdf = sdf.loc[~sdf.responses.isna()]
    sdf.responses = sdf.responses.apply(ast.literal_eval)

if EXPERIMENT == 3:
    demo = sdf.loc[sdf.responses.apply(len) == 3].set_index('pid').responses
    age = demo.apply(get('Q1'))
    gender = demo.apply(get('Q2'))
else:
    demo = sdf.loc[sdf.responses.apply(len) == 2].set_index('pid').responses
    age = demo.apply(get('Q0'))
    gender = demo.apply(get('Q1'))
    
gender = gender.str.lower()
age = age.apply(excepts(ValueError, int, lambda _: None))

write_tex('mean-age', f'{age.mean():.2f}')
write_tex('min-age', str(age.min()))
write_tex('max-age', str(age.max()))

regularize = {
    'man': 'male',
    'woman': 'female',
    'f': 'female',
    'm': 'male',
}
gender = gender.apply(lambda x: regularize.get(x.strip(), x))
write_tex("N-female", str(gender.value_counts()['female']))

write_tex("N-total", f"${len(pdf)}$")
for fb, n in pdf.feedback.value_counts().items():
    write_tex(f"N-{fb}", f"${n}$")

In [ ]:
if EXPERIMENT == 3:
    dropped = pdf.query('not completed').feedback.value_counts()
    rate = dropped / pdf.feedback.value_counts()
    for fb in fb_order:
        write_tex(f'N-drop-{fb}', dropped[fb])
        write_tex(f'drop-rate-{fb}', f'${rate[fb]*100:.1f}\%$')
    write_tex('return-rate', f'${pdf.completed.mean()*100:.1f}\%$')
    write_tex('return-N', f'${pdf.completed.sum()}$')
    pdf.query('completed', inplace=True)
    
pd.Series(pdf.index).to_csv(f'pids/{EXPERIMENT}.csv', index=False, header=False)

In [ ]:
completed = list(pdf.index)
mdf.reset_index(inplace=True)
mdf.query('pid == @completed', inplace=True)
mdf.set_index('pid', inplace=True)

pdf.feedback = pd.Categorical(pdf.feedback, fb_order, ordered=True)    
mdf['feedback'] = pdf.feedback
block_mean = mdf.groupby(['block', 'pid']).score.mean()
for b in ['training', 'test']:
    pdf[b] = block_mean[b]

In [ ]:
# pdf['pay'] = (0.50 + pdf.bonus)
# pdf['wage'] = pdf.pay / (pdf.total_time / 60)

# sns.distplot(pdf.wage)
# plt.axvline(pdf.wage.mean(), c='r', ls='--')

# print(f'The average duration of the experiment was {pdf.total_time.mean():.2f} min +/- {pdf.total_time.std():.2f} min.')
# print(f'The average bonus was ${pdf.bonus.mean():.2f} +/- ${pdf.bonus.std():.2f}.')
# print(f'The average wage was ${pdf.wage.mean():.2f}/hr +/- ${pdf.wage.std():.2f}.')

## Setup Plotting 

In [ ]:
figure = Figures(path=f'figs/{EXPERIMENT}', formats=['pdf', 'png']).plot

sns.set_style('white')
blue, orange = sns.color_palette('tab10')[:2]
gray = (0.5,)*3
red = (1, 0.3, 0.4)
yellow = (1, 0.9, 0.4)
palette = {
    'none': gray,
    'action': blue,
    'meta': orange,
    'info_only': red,
    'reward_only': yellow,
    'both': orange,
}

palette = {
    'none': gray,
    'action': blue,
    'meta': orange,
    'info_only': red,
    'reward_only': yellow,
    'both': orange,
}

nice_names = {
    'meta': 'Metacognitive',
    'action': 'Action',
    'none': 'None',
    'feedback': 'Feedback',
    'info_only': 'Information\nOnly',
    'reward_only': 'Reward\nOnly',
    'both': 'Information &\nReinforcement',
    'score': 'Average Score',
}

def reformat_labels(ax=None):
    ax = ax or plt.gca()
    labels = [t.get_text() for t in ax.get_xticklabels()]
    new_labels = [nice_names.get(lab, lab) for lab in labels]
    ax.set_xticklabels(new_labels)
    
def reformat_legend(ax=None):
    if ax is None:
        ax = plt.gca()
    handles, labels = ax.get_legend_handles_labels()
    print([nice_names.get(l, l).replace('\n', ' ') for l in labels])
    ax.legend(handles=handles, labels=[nice_names.get(l, l).replace('\n', ' ') 
                                       for l in labels])
    
def plot_block_changes():
    block_changes = mdf.loc[1].block.apply(Labeler()).diff().reset_index().query('block == 1').index
    for t in block_changes:
        plt.axvline(t-0.5, c='k', ls='--')

from datetime import datetime
# os.makedirs(f'stats/{EXPERIMENT}/', exist_ok=True)
def result_file(name, ext='tex'):
    file = f'stats/{EXPERIMENT}-{name}.{ext}'
#     with open(file, 'w+') as f:
#         timestamp = datetime.now().strftime('Created on %m/%d/%y at %H:%M:%S\n\n')
#         f.write(timestamp)
    return file

# Test score

In [ ]:
@figure()
def plot_test():
    if EXPERIMENT == 4:
        plt.figure(figsize=(12,4))
    sns.set_style('whitegrid')
    sns.swarmplot('feedback', 'test', data=pdf, palette=palette, alpha=0.5, order=fb_order)
    sns.pointplot('feedback', 'test', data=pdf, palette=palette, order=fb_order, 
                  scale=1, capsize=0.1, markers='o')
    plt.xlabel('Feedback')
    plt.ylabel(nice_names['score'])
    test = 'Test' if EXPERIMENT == 1 else 'Transfer'
    reformat_labels()
    

## Kruskal

In [ ]:
rdf = pdf[['test', 'feedback']].copy()

In [ ]:
%%R -i rdf -o out
out = kruskal.test(test ~ feedback, data=rdf)

In [ ]:
out = dict(out.items())
df = out["parameter"][0]
p = pval(out["p.value"][0])
stat = out["statistic"][0]
write_tex('score-kruskal', rf'$\chi^2({df}) = {stat:.3f}, {p}$')
# p = pvalue(out

## Bootstrapping

### Means

In [ ]:
N_BOOT = 10000
test_score = pdf.set_index('feedback').test

def ci(xs):
    return np.quantile(xs, [0.025, 0.975])

def boot_means(n=N_BOOT):
    r = {}
    for fb in fb_order:
        x = test_score.loc[fb]
        means = [x.sample(frac=1, replace=True).mean() for _ in range(n)]
        r[fb] = np.array(means)
    return r

In [ ]:
bm = boot_means()
for fb in fb_order:
    x = test_score.loc[fb].mean()
    a, b = ci(bm[fb])
    if fb == 'none':
        tex = (rf'${x:.2f}$ points/trial (95\% CI: [${a:.2f}$, ${b:.2f}$])')
    else:
        tex = (rf'${x:.2f}$ points/trial; 95\% CI: [${a:.2f}$, ${b:.2f}$]')
    write_tex(f'mean-{fb}', tex)

### Differences by permutation test

In [ ]:
def null_mean_diff_samples(c1, c2, n=N_BOOT):
    r = []
    x = test_score.copy()
    for i in range(n):
        np.random.shuffle(x.values)
        r.append(x.loc[c1].mean() - x.loc[c2].mean())
    return np.array(r)

#         x = test_score.sample(n=n_fb[c1], replace=True)
#         y = test_score.sample(n=n_fb[c2], replace=True)
#         r.append(x.mean() - y.mean())

def report(c1, c2, n=N_BOOT):
    actual_diff = test_score.loc[c1].mean() - test_score.loc[c2].mean()
    null_diff = null_mean_diff_samples(c1, c2, n=n)
#     p = (1 + np.sum(abs(null_diff) >= abs(actual_diff))) / (n+1)
    p = np.mean(abs(null_diff) >= abs(actual_diff))
    write_tex(f'permutation-{c1}-{c2}', f'${pval(p)}$')

if EXPERIMENT == 4:
    report('both', 'none')
    report('info_only', 'none')
    report('reward_only', 'none')
    report('both', 'reward_only')
    report('both', 'info_only')
else:
    report('meta', 'none')
    if 'action' in fb_order:
        report('meta', 'action')
        report('action', 'none')

# Process

## Summary in test block

In [ ]:
test_block = mdf.query('block == "test"').copy()
first_click = test_block.clicks.apply(get(0, default=None))
test_block['no_click'] = first_click.isna()
test_block['clicked'] = ~test_block.no_click

if EXPERIMENT == 1:
    leaves = {3,4,7,8,11,12}
else:  # transfer
    leaves = {5, 6, 7, 9, 10, 11, 16, 17, 18, 20, 21, 22, 27, 28, 29, 31, 32, 33}
    
x = first_click.isin(leaves)
x[test_block.no_click] = np.nan
test_block['backward'] = x

test_block.backward.fillna(0, inplace=True)

In [ ]:
from scipy.stats import chisquare
rate = test_block.groupby('feedback').n_clicks.mean()

def report_ratio(df, key):
    name = key.replace('_', '-')
    X = df.groupby(['feedback', key]).apply(len)
    rate = 100 * df.groupby('feedback')[key].mean()

    for c in fb_order:
        r = rate[c]
        write_tex(f'{name}-{c}-percent', f"${r:.1f}$\%")

#     for c in 'action', 'none':
#         if c not in fb_order:
#             continue
#         stat, p = chisquare(X['meta'], X[c])
#         tex = rf'$\chi^2(1) = {int(stat)}, {pval(p)}$'
#         write_tex(f'{name}-meta-{c}', tex)

# for c in fb_order:
#     r = rate[c]
#     write_tex(f'clicks-{c}', f"${r:.2f}$")
# report_ratio(test_block, 'no_click')
report_ratio(test_block, 'backward')

## Training

In [ ]:
small_leaves = {3,4,7,8,11,12}
transfer_leaves = {5, 6, 7, 9, 10, 11, 16, 17, 18, 20, 21, 22, 27, 28, 29, 31, 32, 33}

def clicked_leaf_first(row):
    if not row.clicks:
        return False
    first = row.clicks[0]
    transfer =  EXPERIMENT in (2, 3) and row.block == "test"
    leaves = transfer_leaves if transfer else small_leaves
    return first in leaves

mdf['backward'] = mdf.apply(clicked_leaf_first, axis=1)

In [ ]:
from toolz.curried import get
sns.set_style('whitegrid')
@figure()
def plot_backward_training():
#     df = mdf.query('block == "training"').copy()
    df = mdf.copy()
    df.trial_index += 1
    sns.lineplot('trial_index', 'backward', hue='feedback', 
                 data=df, hue_order=fb_order, palette=palette)
    plt.ylabel('Proportion Planning Backward')
    plt.xlabel('Trial Number')
    plt.gca().grid(axis='x')
    split = mdf.query('block == "training"').trial_index.max()
    plt.axvline(split+1.5, c='k', ls='--', alpha=0.3)
#     plt.ylim(0, 1)
    plt.xlim(df.trial_index.min(), df.trial_index.max())
    reformat_legend()

## Mediation

In [ ]:
rdf = test_block.copy().rename(columns={'information': 'info'})
# rdf['feedback'] = (rdf.feedback != 'none').astype(int)

if EXPERIMENT == 4:
    factors = ['info', 'reward']
    for c in factors:
        rdf[c] = rdf[c].astype(float)
else:
    factors = ['action', 'meta']
    fb = rdf.pop('feedback')
    for c in factors:
        rdf[c] = (fb == c).astype(int)

rdf['clicked'] = rdf.clicked.astype(int)
rdf = rdf[[*factors, 'backward', 'clicked', 'n_clicks', 'score', 'trial_index', 'stim_i']].reset_index()
rdf.backward.fillna(0, inplace=True)
# rdf.trial_index = rdf.trial_index.astype(float)
# rdf.query('action == 0', inplace=True)
rdf = rdf.groupby('pid').mean()
rdf.trial_index -= rdf.trial_index.min()

In [ ]:
%%R -i rdf -o score
fit = lm(score ~ backward, data=rdf)
score = summary(fit)
score

In [ ]:
score = dict(score.items())
df = score['df'][1]
est, std, t, p = score['coefficients'][1]
write_tex(f'score-backward', f'${est:.3f}$')
write_tex(f'score-backward-test', f'$t({df}) = {t:.3f}$, ${pval(p)}$')

In [ ]:
%%R -i rdf -i EXPERIMENT -o back -o med_out
library(mediation)

if (EXPERIMENT == 1) {
    back_fit = lm(backward ~ action + meta, data=rdf)
    score_fit = lm(score ~ backward + action + meta, data=rdf)
} else {
    back_fit = lm(backward ~ meta, data=rdf)
    score_fit = lm(score ~ backward + meta, data=rdf)
}
med_out = mediate(back_fit, score_fit, treat="meta", mediator="backward")
back = summary(back_fit)
summary(med_out)

In [ ]:
back = dict(back.items())
df = back['df'][1]
for i, name in enumerate(fb_order[1:], start=1):
    est, std, t, p = back['coefficients'][i]
    write_tex(f'backward-lm-{name}', f'$t({df}) = {t:.3f}$, ${pval(p)}$')

In [ ]:
names = {
    'acme': 'd0',
    'ade': 'z0',
    'total': 'tau',
    'prop': 'n0',
}
med = dict(med_out.items())

for k, v in names.items():
    est = med[v + ('.coef' if v == 'tau' else '')][0]
    lo, hi = med[v+'.ci']
    p = med[v+'.p'][0]
    write_tex(f'mediation-{k}', f'{est:.3f}')
    write_tex(f'mediation-{k}-ci', f'95\% CI: [{lo:.3f}, {hi:.3f}], ${pval(p)}$')

In [ ]:
print("Success!")